In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


In [2]:
from acnportal.acnsim.events import Event, EventQueue, PluginEvent, UnplugEvent, RecomputeEvent
from acnportal.acnsim.interface import Interface
from acnportal.acnsim.models import EV, Battery, EVSE
from acnportal.acnsim.network.sites import caltech_acn, jpl_acn
from acnportal.acnsim.simulator import Simulator
import gym
import numpy as np

from sustaingym.envs.evcharging.actions import get_action_space, to_schedule
from sustaingym.envs.evcharging.ev_charging import EVChargingEnv
from sustaingym.envs.evcharging.event_generation import AbstractTraceGenerator
from sustaingym.envs.evcharging.observations import get_observation_space, get_observation
from sustaingym.envs.evcharging.rewards import get_rewards
from sustaingym.envs.evcharging.utils import MINS_IN_DAY, DATE_FORMAT


In [3]:
acn = caltech_acn()
acn.station_ids[:7]

['CA-308', 'CA-508', 'CA-303', 'CA-513', 'CA-310', 'CA-506', 'CA-316']

In [28]:
class TestGenerator(AbstractTraceGenerator):
    def __init__(self,
                 site: str = "",
                 period: int = 5,
                 recompute_freq: int = 2,
                 date_range = None,
                 requested_energy_cap: float = 100):

        super().__init__(site, period, recompute_freq, date_range, requested_energy_cap)
    
    def update_day(self):
        return
    
    def get_event_queue(self):
        events = []
        for a, d, s in zip([10, 10, 30], [30, 30, 35], ['CA-508', 'CA-303', 'CA-513']):
            req_energy = 30.0
            battery = Battery(capacity=100, init_charge=max(0, 100-req_energy), max_power=100)
            ev = EV(arrival=a, departure=d, requested_energy=req_energy, station_id=s, session_id=str(a), battery=battery, estimated_departure=a+50)

            events.append(PluginEvent(a, ev))
        events.append(RecomputeEvent(0))
        eq = EventQueue(events)
        return eq, 3

In [38]:
tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
eq, _ = tg.get_event_queue()

In [39]:
eq.get_current_events(30)

[acnportal.acnsim.events.event.RecomputeEvent(timestamp=0, event_type=<str object at 0x7f558b640030>, precedence=20),
 acnportal.acnsim.events.event.PluginEvent(timestamp=10, event_type=<str object at 0x7f558b640230>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f558d32e0a0>),
 acnportal.acnsim.events.event.PluginEvent(timestamp=10, event_type=<str object at 0x7f558b640230>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f558d32e6d0>),
 acnportal.acnsim.events.event.PluginEvent(timestamp=30, event_type=<str object at 0x7f558b640230>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f556b5aa610>)]

In [31]:
acn = caltech_acn()
acn.magnitudes

array([ 80.        ,  80.        , 416.66666667, 416.66666667,
       416.66666667, 180.50541516, 180.50541516, 180.50541516])

In [32]:
np.maximum(np.abs(acn.constraint_current(np.ones(shape=(54,), ) * 16, linear=True)) - acn.magnitudes, 0)

array([48., 48.,  0.,  0.,  0.,  0.,  0.,  0.])

In [33]:
(acn.constraint_matrix @ np.ones(shape=(54,), ) * 16)

array([ 128.,  128.,  192., -192.,    0.,   48.,  -96.,   48.])

In [34]:
tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
env = EVChargingEnv(tg, action_type='discrete', verbose=2)
observation = env.reset()


Simulating 3 plug in events. 


In [35]:
print(env.interface._convert_to_amp_periods(30.0, 'CA-508') * 5)
print(env.interface._convert_to_amp_periods(30.0, 'CA-303') * 5)
print(env.interface._convert_to_amp_periods(30.0, 'CA-513') * 5)

8653.846153846154
8653.846153846154
8653.846153846154


In [36]:
from collections import defaultdict

tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
env = EVChargingEnv(tg, action_type='discrete', verbose=2)
for _ in range(1):
    observation = env.reset()
    rewards = 0.
    done = False
    i = 0
    action = np.zeros(shape=(54,), )
    # d = defaultdict(list)
    while not done:
        observation, reward, done, info = env.step(action)
        rewards += reward
        i += 1

    print()
    print()
    print("total iterations: ", i)
    print("environment reward: ", rewards)


Simulating 3 plug in events. 
0
0
-8653.846153846154
-8653.846153846154


total iterations:  4
environment reward:  -17307.69230769231


In [81]:
#-7853.846153846154
#-7053.846153846153

[]

In [185]:
# 0  50 11920.0  12000
# 0 100 13769.23 16000
# 0 150 13769.23 20000
# 0 200 13769.23 24000

11200

11200